In [3]:
import gym
import itertools
import numpy as np
import sys
# if "../" not in sys.path:
#   sys.path.append("../") 
from collections import defaultdict
from lib.envs.cliff_walking import CliffWalkingEnv
from lib.envs.windy_gridworld import WindyGridworldEnv

from scipy.optimize import minimize, rosen, rosen_der
from scipy.optimize import Bounds
from tqdm import tqdm
bounds = Bounds([-0.1,-0.1],[0.1,0.1])

def make_epsilon_greedy_policy(Q, epsilon, nA):
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [2]:
env = CliffWalkingEnv()

# download from https://github.com/Ma-sa-ue/OPE_codes?files=1
Q_space = np.load("Q-table-cliff.npz")["xxx"]
Q_space2 = np.load("Q-table-cliff.npz")["xxx"]

prob1 = [1.0 for i in range((env.nA))]
prob1 = prob1/np.sum(prob1)

###################
## discount rate ##
###################


gamma = 0.5

if gamma == 0.7:
    est_mean = -3.301627552777223
elif gamma == 0.5:
    est_mean = -2.003425807974581
elif gamma == 0.45:
    est_mean = -1.816693355290161
elif gamma == 0.3:
    est_mean = -1.4267969764571544
###################################
## parameter for behavior policy ## larger --> more expert 
###################################
betabeta = 0.5
#betabeta = 0.8
#################################
## parameter for target policy ##
#################################
alpha = 1
#alpha = 0.9
#################################
## parameter for MC repetition ##
#################################

def sample_policy(observation,alpha=alpha):
    prob2 = alpha*Q_space[observation,:] +(1-alpha)*prob1
    return np.random.choice(env.nA,1,p=prob2)[0] ## 4个action 选一
        
def behavior_policy(observation,beta=betabeta):
    prob2 = beta*Q_space[observation,:]+ (1-beta)*prob1 ## behavior这么聪明？
    return np.random.choice(env.nA,1,p=prob2)[0]

def target_dense(observation,alpha=alpha):
    prob2 = alpha*Q_space[observation,:]+ (1-alpha)*prob1
    return prob2

def behav_dense(observation,beta=betabeta):
    prob2 = beta*Q_space[observation,:] + (1-beta)*prob1
    return prob2

In [3]:
def sarsa2(env, policy, policy2, num_episodes, discount_factor=1.0, Q_space2=Q_space2, alpha= 0.6, epsilon=0.03):
    """
    policy : Nothing..
    policy2 : Behavior policy
    """
    print("doing SARSA to get Q prediction")
    
    Q = np.zeros_like(Q_space2)
    episode_episode = []
    for i_episode in tqdm(range(num_episodes)):
        state = env.reset()
        action = policy2(state)
        episode = []
        for t in itertools.count():
            # Take a step
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            # Pick the next action
            next_action= policy2(next_state)
            # TD Update
            td_target = reward + discount_factor * np.sum(Q[next_state,:]*target_dense(next_state)) ## 这个就是 E (Q^\pi)
            td_delta = td_target - Q[state,action]
            Q[state,action] += alpha * td_delta  ## TD update
            if done:
                break
                
            action = next_action
            state = next_state 
        episode_episode.append(episode)
    
    return Q, episode_episode ## return Q and buffers(trajs)

In [6]:
from scipy.stats import norm

def mse(aaa, true = -3.301610434961245):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]
    return [np.mean((((aaa-true)*(aaa-true)))),np.sqrt(np.var((aaa-true)*(aaa-true)))]


rep = 40

sample_size = 1000 * rep
sample_size = int(sample_size/2)


predicted_Q ,episode_episode = sarsa2(env, 
                                      sample_policy,
                                      behavior_policy,
                                      sample_size, 
                                      discount_factor = gamma)

env = env

policy = sample_policy
policy2 = behavior_policy
episode_episode = episode_episode
Q_= predicted_Q
num_episodes= sample_size
discount_factor=gamma


depth = 1


returns_sum = defaultdict(float)
returns_count = defaultdict(float)
returns_count2 = defaultdict(float)
predic_list = []
predic_list2 = []
predic_list3 = []
predic_list22 = []
predic_list4 = []
predic_list5 = np.ones(num_episodes)
auxiauxi = [] 
epiepi = []
weight_list = np.zeros([num_episodes,1000]) ### For bounded IPW
weight_list2 = np.zeros([num_episodes,1002]) ### For bounded IPW
weight_list3 = np.zeros([num_episodes,1002]) ### For bounded IPW
marginal_weight = np.zeros([num_episodes,1000]) ### For bounded IPW
marginal_weight_2 = np.zeros([num_episodes,1000]) ### For bounded IPW
auxi_list = np.zeros([num_episodes,1000])
marginal_auxi_list2 = np.zeros([num_episodes,1000])
marginal_auxi_list = np.zeros([num_episodes,1000])
marginal_auxi_list2_2 = np.zeros([num_episodes,1000])
marginal_auxi_list_2 = np.zeros([num_episodes,1000])
auxi_list2 = np.zeros([num_episodes,1000])
reward_list = np.zeros([num_episodes,1000])
state_list = np.zeros([num_episodes,1000])
action_list = np.zeros([num_episodes,1000])

count_list = np.zeros(1000) 
episolode_longe_list = []

print("doing prediction")
info = []

for i_episode in tqdm(range(num_episodes)):
    episode = episode_episode[i_episode] ## extract episode
    W = 1.0
    W_list = []
    episolode_longe_list.append(len(episode))
    weight_list2[i_episode, 0] = 1.0
    for t in range(len(episode)):
        state, action, reward = episode[t]
        reward_list[i_episode,t] = reward
        state_list[i_episode,t] = state
        action_list[i_episode,t] = action
        if t == 0:
            W = W * target_dense(state)[action]/behav_dense(state)[action]
        else:
            W = W * target_dense(state)[action]/behav_dense(state)[action] * discount_factor

        probprob = alpha * Q_space[state,:] + (1 - alpha) * prob1

        W_list.append(W)
        weight_list[i_episode, t] = W_list[t]
        weight_list2[i_episode, t+1] = W_list[t]
        weight_list3[i_episode, t] = target_dense(state)[action]/behav_dense(state)[action]

        count_list[t] += 1.0

        if t==0:
            auxi_list[i_episode,t] = W_list[t] * Q_[state,action] - np.sum(probprob * Q_[state,:])
        else:
            auxi_list[i_episode,t] = W_list[t] * Q_[state,action]- W_list[t-1] * np.sum(probprob * Q_[state,:])

        if t==0:
            auxi_list2[i_episode,t] = W_list[t]-1.0
        else:
            auxi_list2[i_episode,t] = W_list[t]-W_list[t-1]

# print np.max(np.array(episolode_longe_list))\
# print(episolode_longe_list)
# print(np.array(episolode_longe_list))


weight_list_mean = np.mean(weight_list,1)
reward_list_mean = np.mean(reward_list,1)
auxi_list_mean = np.mean(auxi_list,1)
auxi_list2_mean = np.mean(auxi_list2,1)

val = []    

##### IPW
for i in range(num_episodes):
    predic_list.append(np.sum(weight_list[i,:]*reward_list[i,:]))   
val.append(np.mean(predic_list)) ## val[0]

#### Marginalized-IPW 

for i in range(num_episodes):
    for j in range(episolode_longe_list[i]):
        marginal_weight[i,j] = np.mean(weight_list[:,j][(state_list[:,j]==state_list[i,j]) 
                                                        & (action_list[:,j]==action_list[i,j])])
        if j == 0:
            marginal_weight_2[i,j] = weight_list3[i,j]
        else:
            marginal_weight_2[i,j] = np.mean(weight_list[:,j-1][(state_list[:,j]==state_list[i,j])])*weight_list3[i,j]


for i_episode in tqdm(range(num_episodes)):
    for t in range(episolode_longe_list[i_episode]):
        state = np.int(state_list[i_episode,t])
        action = np.int(action_list[i_episode,t])
        probprob = alpha * Q_space[state,:] + (1 - alpha) * prob1
        if t==0:
            marginal_auxi_list[i_episode,t] = marginal_weight[i_episode,t] * Q_[state,action]- \
                                              np.sum(probprob*Q_[state,:])
            marginal_auxi_list_2[i_episode,t] = marginal_weight_2[i_episode,t] * Q_[state,action]- \
                                                np.sum(probprob*Q_[state,:])
            auxi_list[i_episode,t] = weight_list[i_episode,t]*Q_[state,action]-np.sum(probprob*Q_[state,:])
        else:
#             marginal_auxi_list[i_episode,t] = marginal_weight[i_episode,t]*(Q_[state,action])-marginal_weight[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
#             marginal_auxi_list_2[i_episode,t] = marginal_weight_2[i_episode,t]*(Q_[state,action])-marginal_weight_2[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
#             auxi_list[i_episode,t] = weight_list[i_episode,t]*(Q_[state,action])-weight_list[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
            marginal_auxi_list[i_episode,t] = marginal_weight[i_episode,t]*(Q_[state,action])- \
                                discount_factor * marginal_weight[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
            marginal_auxi_list_2[i_episode,t] = marginal_weight_2[i_episode,t]*(Q_[state,action])-\
                                discount_factor * marginal_weight_2[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))
            auxi_list[i_episode,t] = weight_list[i_episode,t]*(Q_[state,action])- \
                                discount_factor * weight_list[i_episode,t-1]*np.sum(probprob*(Q_[state,:]))

        if t==0:
            marginal_auxi_list2[i_episode,t] = marginal_weight[i_episode,t]-1.0
            marginal_auxi_list2_2[i_episode,t] = marginal_weight_2[i_episode,t]-1.0
            auxi_list2[i_episode,t] = weight_list[i_episode,t]-1.0
        else:
            marginal_auxi_list2[i_episode,t] =  marginal_weight[i_episode,t]- marginal_weight[i_episode,t-1]
            marginal_auxi_list2_2[i_episode,t] =  marginal_weight_2[i_episode,t]- marginal_weight_2[i_episode,t-1]
            auxi_list2[i_episode,t] = weight_list[i_episode,t]-weight_list[i_episode,t-1]


for i in range(num_episodes):
    predic_list2.append(np.sum(marginal_weight[i,:]*reward_list[i,:]))   

### marginal ipw2  #### Using action and state 
val.append(np.mean(predic_list2))  ## val[1]


### marginal ipw3#### Using only state 
for i in range(num_episodes):
    predic_list22.append(np.sum(marginal_weight_2[i,:]*reward_list[i,:]))    ## val[2]

val.append(np.mean(predic_list22))


#### DR ## check
val.append(np.mean(predic_list) - np.mean(np.sum(auxi_list,1)))  # DRL(MDP)

#### marginal DR 1  #### Using action and state 
val.append(np.mean(predic_list2) - np.mean(np.sum(marginal_auxi_list,1)))     # DRL(NMDP, lag=3)
#### marginal DR 2   #### Using only state                                     
val.append(np.mean(predic_list22) - np.mean(np.sum(marginal_auxi_list_2,1)))  # DRL(NMDP, lag=3)


  0%|          | 24/20000 [00:00<01:39, 199.97it/s]

doing SARSA to get Q prediction


  0%|          | 76/20000 [00:00<00:26, 758.78it/s]

doing prediction


100%|██████████| 20000/20000 [00:36<00:00, 548.91it/s]


In [7]:


## output
wis_output  = predic_list
ipw3_output = predic_list22
dr_output = predic_list - np.sum(auxi_list,1)
dr2_output = predic_list2 - np.sum(marginal_auxi_list,1)
dr3_output = predic_list22 - np.sum(marginal_auxi_list_2,1)

for method, output in zip(["wis_output", "ipw3_output", "dr_output", "dr2_output", "dr3_output"], 
                          [wis_output, ipw3_output, dr_output, dr2_output, dr3_output]):
    
    count = 0
    pred_output = []
    AL = []
    alpha = 0.05

    for i in range(rep):
        n = int(sample_size/rep)
        V_output = output[i * n  : (i + 1) * n]
        lower_bound = np.mean(V_output) - norm.ppf(1 - alpha/2) * np.std(V_output)/(n**0.5)
        upper_bound = np.mean(V_output) + norm.ppf(1 - alpha/2) * np.std(V_output)/(n**0.5)
        #lower_bound, upper_bound, upper_bound - lower_bound, len(V_output)
        if lower_bound < est_mean and est_mean < upper_bound:
            count += 1
        AL.append(upper_bound - lower_bound)
        pred_output.append((lower_bound + upper_bound) / 2)
    print("%s: CI %.3f, AL %.3f, pred %.3f, MSE %.4f(%.4f)" %(method, count / len(AL),  np.mean(AL), np.mean(pred_output), *mse(pred_output, est_mean)))

wis_output: CI 0.975, AL 0.376, pred -2.012, MSE 0.0090(0.0109)
ipw3_output: CI 0.000, AL 0.633, pred -3.016, MSE 1.0485(0.2889)
dr_output: CI 1.000, AL 0.182, pred -2.011, MSE 0.0017(0.0016)
dr2_output: CI 1.000, AL 0.182, pred -2.011, MSE 0.0017(0.0016)
dr3_output: CI 0.000, AL 0.325, pred -2.629, MSE 0.3955(0.0819)


In [7]:
print("wis", np.mean(predic_list), np.std(predic_list))
print("ipw3", np.mean(predic_list22), np.std(predic_list22))
print("dr", np.mean(predic_list - np.sum(auxi_list,1)), np.std(predic_list - np.sum(auxi_list,1)))
print("dr3", np.mean(predic_list22 - np.sum(marginal_auxi_list_2,1)), np.std(predic_list22 - np.sum(marginal_auxi_list_2,1)))

wis_output  = predic_list
ipw3_output = predic_list22
dr_output = predic_list - np.sum(auxi_list,1)
dr3_output = predic_list22 - np.sum(marginal_auxi_list_2,1)

wis -1.9878388767460762 2.1234158255288036
ipw3 -2.946639298358889 3.580992582049927
dr -2.00367294979227 0.8599963829612072
dr3 -1.7500946343754875 1.3391783883898343


In [17]:

def mse(aaa, true = -3.301610434961245):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]
    return [np.mean((((aaa-true)*(aaa-true)))),np.sqrt(np.var((aaa-true)*(aaa-true)))]

print("dr3", mse(dr3_output))
print("dr", mse(dr_output))
print("ipw3", mse(ipw3_output))
print("wis", mse(wis_output))

dr3 [0.16089610591722953, 1.1053714990773462]
dr [0.08631137838797087, 0.5508526551595504]
ipw3 [53.997189444281005, 277.1743438097648]
wis [27.35327656291926, 131.55643170999835]


In [18]:
from scipy.stats import norm

alpha = 0.05
n = sample_size
V_output = dr3_output
lower_bound = np.mean(V_output) - norm.ppf(1 - alpha/2) * np.std(V_output)/(n**0.5)
upper_bound = np.mean(V_output) + norm.ppf(1 - alpha/2) * np.std(V_output)/(n**0.5)
lower_bound, upper_bound, upper_bound - lower_bound, len(V_output)

(-3.329633795201816, -3.259327011705076, 0.07030678349673991, 500)

In [41]:
# true = -42.49
# def tablize_result(aaa):
#     aaa = np.array(aaa)
#     # aaa = aaa[aaa>-100]
#     rmse_value = np.sqrt(np.mean(np.square(aaa - true)))
#     # sd_value = np.std(np.square(aaa - true))
#     sd_value = np.std(aaa - true)
#     return [rmse_value, sd_value]

# print("is")
# print(tablize_result(is_list))
# print("dr3")
# print(tablize_result(dr3_list))
# print("dm")
# print(tablize_result(dm_list))
# print("mis")
# print(tablize_result(is3_list))
# print("dr")
# print(tablize_result(dr_list))

In [42]:
is_list

[-3.202400967458164]

In [150]:
true = -42.58326666666667
def mse(aaa):
    aaa = np.array(aaa)
    aaa = aaa[aaa>-100]
    return [np.mean((((aaa-true)*(aaa-true)))),np.sqrt(np.var((aaa-true)*(aaa-true)))]

print("is")
print(np.mean(is_list))
print(mse(is_list))
print("wis")
print(np.mean(is3_list))
print(mse(is3_list))
print("dm")
print(np.mean(dm_list))
print(mse(dm_list))
print("dr")
print(np.mean(dr_list))
print(mse(dr_list))
print("dr2")
print(np.mean(dr2_list))
print(mse(dr2_list))
print("dr3")
print(np.mean(dr3_list))
print(mse(dr3_list))

In [240]:
predicted_Q ,episode_episode = sarsa2(env, sample_policy, sample_policy, 3000, discount_factor= gamma)

gamma = gamma
cum_reward_list = []
for i in range(len(episode_episode)):
    t = 0
    cum_reward = 0
    for episode in episode_episode[i]:
        cum_reward += episode[-1] * gamma ** t
        t += 1
    cum_reward_list.append(cum_reward )   
    # print(t)
np.mean(cum_reward_list), np.std(cum_reward_list)

  1%|          | 24/3000 [00:00<00:12, 234.65it/s]

doing SARSA to get Q prediction


100%|██████████| 3000/3000 [00:07<00:00, 423.44it/s]


(-3.3010369965309994, 4.440892098500626e-16)